# Ranking Mundial - EXP e IMP Agro

Fonte: [FAO - ONU](http://www.fao.org/faostat/en/#data/TP)

## Importação de bibliotecas

In [50]:
# download dados
import requests
from io import BytesIO
from zipfile import ZipFile

# manipulacao
import pandas as pd
import numpy as np

# graphs
import altair as alt


## Download dados

In [41]:
#aprox 61 MB
data_path = "http://fenixservices.fao.org/faostat/static/bulkdownloads/Trade_Crops_Livestock_E_All_Data.zip"
download = requests.get(data_path).content

In [45]:
# descompactando
zipfile = ZipFile(BytesIO(download))
zipfile.filelist

[<ZipInfo filename='Trade_Crops_Livestock_E_All_Data.csv' compress_type=deflate external_attr=0x20 file_size=294672016 compress_size=31955939>,
 <ZipInfo filename='Trade_Crops_Livestock_E_All_Data_NOFLAG.csv' compress_type=deflate external_attr=0x20 file_size=225334400 compress_size=30100016>,
 <ZipInfo filename='Trade_Crops_Livestock_E_Flags.csv' compress_type=deflate external_attr=0x20 file_size=275 compress_size=186>]

O arquivo baixado é composto por 3 outros arquivos. Escolhemos trabalhar com o segundo.

In [63]:
df = pd.read_csv(zipfile.open(zipfile.filelist[1]), encoding='latin1')

In [64]:
print(df.shape)
df.head()

(378363, 66)


,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y1961,Y1962,Y1963,...,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019
0,2,Afghanistan,862,Alfalfa meal and pellets,5910,Export Quantity,tonnes,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.0
1,2,Afghanistan,862,Alfalfa meal and pellets,5922,Export Value,1000 US$,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0
2,2,Afghanistan,231,Almonds shelled,5610,Import Quantity,tonnes,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,167.0,181.0,845.0,1786.0,597.0,3520.0
3,2,Afghanistan,231,Almonds shelled,5622,Import Value,1000 US$,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1116.0,1377.0,4988.0,11047.0,1242.0,25540.0
4,2,Afghanistan,231,Almonds shelled,5910,Export Quantity,tonnes,0.0,0.0,0.0,...,1308.0,2262.0,1125.0,2381.0,2716.0,2342.0,1691.0,3335.0,3339.0,6848.0


## Filtro e preparação dos dados

Apesar de listar dados desde 1961, esses dados estão presentes em poucos os casos.

Vamos focar o restante da análise somente no dado mais recente (2019)

In [65]:
df = df[['Area Code', 'Area', 'Item Code', 'Item', 'Element Code', 'Element',
       'Unit', 'Y2019']]

Os dados compreendem não só países mas também grupos de países definidos por critérios geográficos ou econômicos.

Para essa análise, removemos os grupos e prosseguimos somente com os países.

In [66]:
exclude_areas = [
    'World',
    # continentes e subcontinentes
    'Oceania', 'Oceania (excluding intra-trade)',
    'Australia and New Zealand', 'Australia and New Zealand (excluding intra-trade)',
    #
    'Europe', 'Europe (excluding intra-trade)',
    'Eastern Europe', 'Eastern Europe (excluding intra-trade)',
    'Northern Europe', 'Northern Europe (excluding intra-trade)',
    'Western Europe', 'Western Europe (excluding intra-trade)',
    'Southern Europe', 'Southern Europe (excluding intra-trade)',
    'Belgium-Luxembourg',
    #
    'Asia', 'Asia (excluding intra-trade)',
    'Central Asia', 'Central Asia (excluding intra-trade)',
    'Eastern Asia', 'Eastern Asia (excluding intra-trade)', 
    'Western Asia', 'Western Asia (excluding intra-trade)',
    'Southern Asia', 'Southern Asia (excluding intra-trade)',
    'South-eastern Asia', 'South-Eastern Asia (excluding intra-trade)',
    'USSR',
    'China (excluding intra-trade)',
    'China, Hong Kong SAR',
    'China, Macao SAR',
    'China, Taiwan Province of',
    'China, mainland',
    'Melanesia', 'Melanesia (excluding intra-trade)',
    'Micronesia', 'Micronesia (excluding intra-trade)', 
    'Polynesia', 'Polynesia (excluding intra-trade)',
    #
    'Americas', 'Americas (excluding intra-trade)'
    'Northern America', 'Northern America (excluding intra-trade)', 
    'Central America', 'Central America (excluding intra-trade)',
    'South America', 'South America (excluding intra-trade)',
    'Caribbean', 'Caribbean (excluding intra-trade)',
    #
    'Africa', 'Eastern Africa',
    'Africa (excluding intra-trade)',
    'Northern Africa', 'Northern Africa (excluding intra-trade)', 
    'Middle Africa', 'Middle Africa (excluding intra-trade)',
    'Eastern Africa (excluding intra-trade)', 
    'Western Africa', 'Western Africa (excluding intra-trade)',
    'Southern Africa', 'Southern Africa (excluding intra-trade)',
    'Sudan (former)',  
    # classificacoes e grupos economicos
    'Least Developed Countries', 'Least Developed Countries (excluding intra-trade)',
    'Low Income Food Deficit Countries', 'Low Income Food Deficit Countries (excluding intra-trade)',
    'Net Food Importing Developing Countries', 'Net Food Importing Developing Countries (excluding intra-trade)',
    'Small Island Developing States', 'Small Island Developing States (excluding intra-trade)',
    'European Union (12) (excluding intra-trade)',
    'European Union (15) (excluding intra-trade)',
    'European Union (25) (excluding intra-trade)',
    'European Union (27)', 'European Union (27) (excluding intra-trade)',
    'European Union (28)', 'European Union (28) (excluding intra-trade)',
    'Land Locked Developing Countries', 'Land Locked Developing Countries (excluding intra-trade)',
]

In [71]:
# exclui os grupos e os não disponíveis
df = df[~df['Area'].isin(exclude_areas)].dropna()

Ainda uma outra separação é necessária.

Os items foram listados individualmente e agregados. Abaixo, separamos esses dois tipos.

In [72]:
group_products = [
    'Agricultural Products', 'Food Excluding Fish', 'Cereals and Preparations',
    'Cereal preparations', 'Cereals', 'Fats and Oils (excluding Butter)',
    'Animal Fats and Oils (excl. Butter)', 'Vegetable Oil and Fat',
    'Meat and Meat Preparations', 'Bovine Meat', 'Poultry Meat',
    'Pigmeat', 'Other Meat', 'Sugar and Honey', 'Fruit and Vegetables',
    'Fruit', 'Vegetables', 'Pulses', 'Roots and Tubers', 'Nuts',
    'Sugar Crops Primary', 'Dairy Products and Eggs', 'Dairy Products',
    'Eggs', 'Beverages', 'Alcoholic Beverages', 'Non-alcoholic Beverages',
    'Other food', 'Oilseeds', 'Other food nes', 'Non-food',
    'Fodder and Feeding Stuff', 'Non-edible Crude Materials',
    'Natural Rubber', 'Textile Fibres', 'Hides and skins',
    'Crude Materials nes', 'Non-edible Fats and Oils', 'Tobacco',
    'Total Merchandise Trade',
]

In [73]:
# separação
df_produtos = df[~df['Item'].isin(group_products)]
df_agregados = df[df['Item'].isin(group_products)]

In [74]:
df_produtos.head()

,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y2019
0,2,Afghanistan,862,Alfalfa meal and pellets,5910,Export Quantity,tonnes,49.0
1,2,Afghanistan,862,Alfalfa meal and pellets,5922,Export Value,1000 US$,18.0
2,2,Afghanistan,231,Almonds shelled,5610,Import Quantity,tonnes,3520.0
3,2,Afghanistan,231,Almonds shelled,5622,Import Value,1000 US$,25540.0
4,2,Afghanistan,231,Almonds shelled,5910,Export Quantity,tonnes,6848.0


In [75]:
def separa_tipos(dataf):
    exp_value = dataf[dataf['Element'] == "Export Value"]
    exp_qtd = dataf[dataf['Element'] == "Export Quantity"]
    imp_value = dataf[dataf['Element'] == "Import Value"]
    imp_qtd = dataf[dataf['Element'] == "Import Quantity"]
    return exp_value, exp_qtd, imp_value, imp_qtd

In [111]:
exp_value_prod, exp_qtd_prod, imp_value_prod, imp_qtd_prod = separa_tipos(df_produtos)
exp_value_grupo, exp_qtd_grupo, imp_value_grupo, imp_qtd_grupo = separa_tipos(df_agregados)

## Criação dos Rankings

In [116]:
def adiciona_ranking(dados):
    dados = dados.assign(
        rank = dados
        .groupby("Item")["Y2019"]
        .rank("dense", ascending=False)
    )
    return dados

#
exp_value_prod = adiciona_ranking(exp_value_prod)
exp_qtd_prod = adiciona_ranking(exp_qtd_prod)
imp_value_prod = adiciona_ranking(imp_value_prod)
imp_qtd_prod = adiciona_ranking(imp_qtd_prod)
#
exp_value_grupo = adiciona_ranking(exp_value_grupo)
exp_qtd_grupo = adiciona_ranking(exp_qtd_grupo)
imp_value_grupo = adiciona_ranking(imp_value_grupo)
imp_qtd_grupo = adiciona_ranking(imp_qtd_grupo)

## Produtos em que o país está no Top 5

In [120]:
def produtos_top_n_pais(dados, n, pais):
    topn = dados[dados["rank"] <= n]
    return topn[(topn['Area'] == pais) &
                (topn['Y2019'] != 0)]

### Exportação em Valores

In [121]:
top5_exp_value_prod = produtos_top_n_pais(exp_value_prod, 5, "Brazil")
top5_exp_value_prod

,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y2019,rank
30852,21,Brazil,245,"Cake, groundnuts",5922,Export Value,1000 US$,3643.0,3.0
30876,21,Brazil,238,"Cake, soybeans",5922,Export Value,1000 US$,5855001.0,3.0
30892,21,Brazil,230,"Cashew nuts, shelled",5922,Export Value,1000 US$,121174.0,5.0
31002,21,Brazil,659,"Coffee, extracts",5922,Export Value,1000 US$,582539.0,4.0
31006,21,Brazil,656,"Coffee, green",5922,Export Value,1000 US$,4575024.0,2.0
31028,21,Brazil,767,Cotton lint,5922,Export Value,1000 US$,2640378.0,4.0
31032,21,Brazil,770,Cotton linter,5922,Export Value,1000 US$,10844.0,5.0
31120,21,Brazil,628,"Feed, pulp of fruit",5922,Export Value,1000 US$,87421.0,2.0
31128,21,Brazil,821,Fibre crops nes,5922,Export Value,1000 US$,44003.0,3.0
31132,21,Brazil,569,Figs,5922,Export Value,1000 US$,6709.0,5.0


### Exportação em Quantidade

In [122]:
top5_exp_qtd_prod = produtos_top_n_pais(exp_qtd_prod, 5, "Brazil")
top5_exp_qtd_prod

,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y2019,rank
30875,21,Brazil,238,"Cake, soybeans",5910,Export Quantity,tonnes,16681651.0,3.0
31001,21,Brazil,659,"Coffee, extracts",5910,Export Quantity,tonnes,99350.0,5.0
31005,21,Brazil,656,"Coffee, green",5910,Export Quantity,tonnes,2230872.0,2.0
31027,21,Brazil,767,Cotton lint,5910,Export Quantity,tonnes,1613670.0,4.0
31031,21,Brazil,770,Cotton linter,5910,Export Quantity,tonnes,23410.0,5.0
31119,21,Brazil,628,"Feed, pulp of fruit",5910,Export Quantity,tonnes,195978.0,2.0
31127,21,Brazil,821,Fibre crops nes,5910,Export Quantity,tonnes,36150.0,5.0
31151,21,Brazil,126,"Flour, cassava",5910,Export Quantity,tonnes,11483.0,3.0
31207,21,Brazil,626,"Fruit, cooked, homogenized preparations",5910,Export Quantity,tonnes,12431.0,5.0
31339,21,Brazil,498,"Juice, lemon, single strength",5910,Export Quantity,tonnes,9870.0,1.0


### Importação em Valores

In [123]:
top5_imp_value_prod = produtos_top_n_pais(imp_value_prod, 5, "Brazil")
top5_imp_value_prod

,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y2019,rank
30882,21,Brazil,101,Canary seed,5622,Import Value,1000 US$,15346.0,1.0
31224,21,Brazil,406,Garlic,5622,Import Value,1000 US$,225095.0,4.0
31414,21,Brazil,49,Malt,5622,Import Value,1000 US$,543581.0,1.0
31702,21,Brazil,402,"Onions, shallots, green",5622,Import Value,1000 US$,52475.0,1.0


### Importação em Quantidade

In [124]:
top5_imp_qtd_prod = produtos_top_n_pais(imp_qtd_prod, 5, "Brazil")
top5_imp_qtd_prod

,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y2019,rank
30773,21,Brazil,176,"Beans, dry",5610,Import Quantity,tonnes,150660.0,4.0
30881,21,Brazil,101,Canary seed,5610,Import Quantity,tonnes,29114.0,1.0
30893,21,Brazil,217,"Cashew nuts, with shell",5610,Import Quantity,tonnes,5048.0,5.0
31223,21,Brazil,406,Garlic,5610,Import Quantity,tonnes,165446.0,3.0
31413,21,Brazil,49,Malt,5610,Import Quantity,tonnes,1091558.0,1.0
31693,21,Brazil,262,Olives preserved,5610,Import Quantity,tonnes,121525.0,4.0
31701,21,Brazil,402,"Onions, shallots, green",5610,Import Quantity,tonnes,211524.0,1.0
31721,21,Brazil,521,Pears,5610,Import Quantity,tonnes,153489.0,4.0
31769,21,Brazil,537,Plums dried (prunes),5610,Import Quantity,tonnes,11633.0,2.0
31817,21,Brazil,28,"Rice, husked",5610,Import Quantity,tonnes,262231.0,3.0
